In [1]:
using CSV, DataFrames, Random, LinearAlgebra, Distances, Distributions, SpecialFunctions, Plots


#Calculate the probabilities of getting an offspring with trait k when the gametes with 'trait 
#value' i an j are combined

function qgprob(n::Int64)
    
    #All possible phenotypes
    pheno= collect(1:(2*n+1)) ./ (2*n+1)
    nt=length(pheno)

    G=zeros(Float64,n+1,n+1,n+1)

    for i in 0:n, j in 0:i, k in max(0,(i+j-n)):min(n,(i+j))
                m=collect(0:min(j,k,i+j-k))
                G[1+i,1+j,1+k]=sum(pdf.(Hypergeometric(i,n-i,j),m).*pdf.(Binomial.(i+j .- (2 .* m)),k .- m))
    end

    for k in 0:n
        G[:,:,1+k]=G[:,:,1+k]+transpose(G[:,:,1+k])
        for i1 in 0:n
            G[i1+1,i1+1,k+1] /= 2
        end
    end

    ind_haplR=zeros(Float64,2*n+1, 2*n+1)

    for k in 0:n
        for i in 0:n
             ind_haplR[1+i,1+k] = G[1+i,1,1+k]
            for j in 0:n
                ind_haplR[1+j+n,1+k]=G[1+n,1+j,1+k]
            end
        end
    end

    R=zeros(Float64,nt,nt,nt)

    for i in 0:(2*n), j in 0:(2*n), q in 0:(2*n)
         R[1+i,1+j,1+q]= sum(ind_haplR[1+i,1 .+ (0:q)] .* 
                             ind_haplR[1+j,1+q .- (0:q)])
    end
        
    return R
end
    
    
#Function to simulate population dynamics
    
function multipop(Ns::Vector{Float64},r::Vector{Float64},K::Vector{Float64},R::Array{Float64},
                    mmat::Matrix{Float64},pa::Float64,SST::Matrix{Float64})
        
    tsteps=size(SST)[1]
        
    nloci=(size(R)[1]-1)/2
    pheno= collect(1:(2*nloci+1)) ./ (2*nloci+1)
        
    N=rand(Uniform(0,1.0),length(Ns),size(R)[1])
    Ng0= N ./ sum(eachcol(N))
    Np0= Ng0 .* Ns 
    Ngen=deepcopy(Ng0)
    Np=deepcopy(Np0)

    #Set up optimal traits value change with time using SST data
    #Initial value is the population mean (this assumes that the local populations are 
    #already adapted to the local temperature).
        
    topt0= [sum(pheno .* Np[x,:])/sum(Np[x,:]) for x in 1:size(Np)[1]]

    topts=topt0 .+ (1.2 .* SST')
        
    #precalculate selection pressure on phenotypes across time steps
    probsurv=zeros(Float64, tsteps, length(Ns), length(pheno))
        
    [probsurv[x,y,:]=exp.(-((topts[y,x] .-pheno) .^2) ./ 0.16) for x in 1:tsteps, y in 1:length(Ns) ]
     
    #Store results
    res=zeros(Float64,tsteps+1,length(Ns),length(pheno))

    res[1,:,:]=Np0

    for t in 1:tsteps
            
        if sum(floor.(Np))<1
                break
        
        else

            #Selection event
            
            Np=Np .* (probsurv[t,:,:])

            #Reproduction event: Apply pop gen model and readjust phenotype frequencies of the 
            #next generation

            if(pa<1)

                newgen=zeros(Float64,length(Ns),length(pheno))

                for i in 1:size(Ngen)[1]
                    
                    Ngen[i,:]=Np[i,:]/sum(Np[i,:])

                    probs=Ngen[i,:]*(Ngen[i,:]')

                    for j in 1:size(Ngen)[2]

                        newgen[i,j]=sum(probs.*R[:,:,j])
                    end

                    Np[i,:]=((pa*sum(Np[i,:])) .* Ngen[i,:]) + (((1-pa)*sum(Np[i,:])) .* newgen[i,:])  

                end
            end

            #Settlement: apply the growth rate and crowding effect

            for i in 1:size(Np)[1]
                    
                #Logistic growth to simulate crowding effect during settlement
                Ngen[i,:]=Np[i,:]/sum(Np[i,:])
                                                        
                Np[i,:]=(sum(Np[i,:])+ (r[i]*sum(Np[i,:])*(1-(sum(Np[i,:])/K[i])))) .* Ngen[i,:]

            end

            #Migration
            #add fraction of populations to each other (uniformly across all phenotypes) according to
            #the migration matrix mmat
            for i in 1:size(Np)[1], j in 1:size(Np)[1]

                Np[i,:] += mmat[i,j] .* Np[j,:]
                
            end
            
            Np[Np .<1 ] .= 0
            
            res[t+1,:,:]=Np

        end
    end
    
    return res
        
end

[ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Cairo_jll [83423d85-b0ee-5818-9007-b63ccbeb887a]
│   exception = Required dependency Fontconfig_jll [a3f928ae-7b40-5064-980b-68af3947d34b] failed to load from a cache file.
└ @ Base loading.jl:1818
[ Info: Skipping precompilation since __precompile__(false). Importing Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80].
[ Info: Precompiling FFMPEG [c87230d0-a227-11e9-1b43-d7ebe4e7570a]
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Cairo_jll [83423d85-b0ee-5818-9007-b63ccbeb887a]
│   exception = Required dependency Fontconfig_jll [a3f928ae-7b40-5064-980b-68af3947d34b] failed to load from a cache file.
└ @ Base loading.jl:1818
[ Info: Skipping precompilation since __precompile__(false). Importing FFMPEG [c87230d0-a227-11e9-1b43-d7ebe4e7570a].
[ Info: Precompiling FFMPEG_jll [b22a6f82-2f65-5046-a5b2-3

multipop (generic function with 1 method)